In [56]:
import os
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# all directories
TRAIN_DIR = 'extracted/cats_and_dogs_filtered/train'
VAL_DIR = 'extracted/cats_and_dogs_filtered/validation'
TRAIN_DIR_CATS = 'extracted/cats_and_dogs_filtered/train/cats'
TRAIN_DIR_DOGS = 'extracted/cats_and_dogs_filtered/train/dogs'
VAL_DIR_CATS = 'extracted/cats_and_dogs_filtered/validation/cats'
VAL_DIR_DOGS = 'extracted/cats_and_dogs_filtered/validation/dogs'

# getting images squared
imageSize = 200

# alpha
lr = 0.001

# momentum
beta_1 = 0.9

# mini-batch size
BATCH_SIZE = 32

# number of epochs
epochs = 10

# total training data and val data
totalTrain = len(os.listdir(TRAIN_DIR_CATS))
totalTrain += len(os.listdir(TRAIN_DIR_DOGS))

totalVal = len(os.listdir(VAL_DIR_CATS))
totalVal += len(os.listdir(VAL_DIR_DOGS))

In [57]:
# for data augmentation since we have a small training set
trainImageGenerator = ImageDataGenerator(
                                        rescale = 1./255.,  # normalization
                                        rotation_range = 40,  # range of rotation
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        shear_range = 0.2,
                                        zoom_range = 0.2,
                                        horizontal_flip = True,  # fliiping images horizontally
                                        fill_mode = 'nearest'  # any pixel gaps will be filled with the 'nearest' pizel
                                        )

validationImageGenerator = ImageDataGenerator(
                                        rescale = 1./255.
                                        )

In [58]:
# declaring training data
trainingData = trainImageGenerator.flow_from_directory(
                                        batch_size = BATCH_SIZE,
                                        directory = TRAIN_DIR,
                                        shuffle = True,
                                        target_size = (imageSize, imageSize),
                                        class_mode = 'binary'   # only cats or dogs, binary classification
                                        )

validationData = validationImageGenerator.flow_from_directory(
                                        batch_size = BATCH_SIZE,
                                        directory = VAL_DIR,
                                        shuffle = False,
                                        target_size = (imageSize, imageSize),
                                        class_mode = 'binary'
                                        )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [59]:
# printing classes
print(trainingData.class_indices)

{'cats': 0, 'dogs': 1}


In [68]:
# clearing previous models
K.backend.clear_session()

# defining the model
model = K.models.Sequential([
    Conv2D(filters = 32, kernel_size = (3, 3), input_shape = (imageSize, imageSize, 3)), # 3 for RGB
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    Conv2D(filters = 64, kernel_size = (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    Conv2D(filters = 128, kernel_size = (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    Conv2D(filters = 128, kernel_size = (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    Conv2D(filters = 128, kernel_size = (3, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
#     Conv2D(8, (1, 1)),
#     BatchNormalization(),
    
    Flatten(),
    
    Dense(units = 512, activation = 'relu'),
    Dropout(rate = 0.5),        # regularization to reduce variance
    
    Dense(units = 2, activation = 'softmax')
    
], name = 'cats-vs-dogs')

In [69]:
# compiling the model
opt = K.optimizers.Adam(lr = lr, beta_1 = beta_1)
model.compile (
    optimizer = opt,
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
    )

In [70]:
# summary of the model
model.summary()

Model: "cats-vs-dogs"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 198, 198, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 97, 97, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 97, 97, 64)       

In [71]:
# fit is overloaded with fit_generator in TensorFlow 2.2+, hence, using fit
history = model.fit(
    trainingData,
    steps_per_epoch = totalTrain // BATCH_SIZE,  # steps per epoch is defined as total by batch size
    epochs = epochs,
    validation_data = validationData,
    validation_steps = totalVal // BATCH_SIZE
    )

Epoch 1/10
62/62 [==============================] - 240s 4s/step - loss: 1.4002 - accuracy: 0.5381 - val_loss: 0.7043 - val_accuracy: 0.5040
Epoch 2/10
16/62 [======>.......................] - ETA: 2:21 - loss: 0.6864 - accuracy: 0.5444

KeyboardInterrupt: 

In [105]:
model.evaluate(validationData, batch_size = BATCH_SIZE)

32/32 [==============================] - 3s 97ms/step - loss: 0.6429 - accuracy: 0.6180


[0.6428672671318054, 0.6179999709129333]

In [104]:
# saving json format of model
modelJSON = model.to_json()
with open('cats_vs_dogs.json', 'w') as file:
    file.write(modelJSON)
    
# saving weights of model
model.save_weights("cats_vs_dogs_weights.h5")

#printing confirmation
print("Saved model to disk")

Saved model to disk


In [106]:
# loading model
jsonFile = open('cats_vs_dogs.json', 'r')
loadedJSONModel = jsonFile.read()
jsonFile.close()
loadedModel = K.models.model_from_json(loadedJSONModel)

# loading weights using this model
loadedModel.load_weights('cats_vs_dogs_weights.h5')

In [108]:
# compiling the loaded model
loadedModel.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

# evaluating the loaded model
loadedModel.evaluate(validationData, batch_size = BATCH_SIZE)

32/32 [==============================] - 3s 95ms/step - loss: 0.6429 - accuracy: 0.6180


[0.6428672671318054, 0.6179999709129333]